### Python import

In [ ]:
import os
import cv2
import sys
import numpy as np
import pandas as pd
from scipy.stats import mode
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

import torch
import torchvision
import albumentations as A
from torch import nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from adabelief_pytorch import AdaBelief

from module.config import config_from_yaml
from module.helper import print_time, AverageMeter
from module.coco import coco_dataset
from module.image import imread
from module.label_map import get_label_map
from module.dataset import MASK_Dataset
from module.model import MASK_MODEL
from module.engine import evaluate, print_mAP
from module.scheduler import CosineAnnealingWarmupRestarts

CFG = config_from_yaml('config.yaml')

### Transform

In [ ]:
train_transform = A.Compose([
    A.Resize(CFG.TRAIN.IMG_SIZE, CFG.TRAIN.IMG_SIZE, p=1),
    A.ImageCompression(quality_lower=99, quality_upper=100, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
    A.ShiftScaleRotate(shift_limit=0, scale_limit=[-0.5, 0.0], rotate_limit=0, interpolation=0, border_mode=0, p=1.0),
],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

valid_transform = A.Compose([
    A.Resize(CFG.TRAIN.IMG_SIZE, CFG.TRAIN.IMG_SIZE, p=1),
], 
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

### Sampler

In [ ]:
def make_sampler(dataset):
    count   = np.bincount(dataset.labels).tolist()
    weights = 1./torch.tensor(count, dtype=torch.float)
    weights = weights[dataset.labels]
    sampler = WeightedRandomSampler(weights=weights, num_samples=len(weights))
    return sampler

def collate_fn(batch):
    return tuple(zip(*batch))

### Data

In [ ]:
# All Data
np.random.seed(CFG.BASE.SEED)

df        = pd.read_csv(CFG.DATA.PSEUDO_LABEL_FILE)
df_tmp    = df[['img_name', 'defect']].drop_duplicates()
img_names = df_tmp['img_name'].values
defects   = df_tmp['defect'].values

train_img_names = img_names
valid_img_names = np.sort(np.random.choice(img_names, size=4000))
coco_dataset(df, 'train.json', img_names=train_img_names)
coco_dataset(df, 'valid.json', img_names=valid_img_names)

train_dataset = MASK_Dataset('train.json', transforms=train_transform)
valid_dataset = MASK_Dataset('valid.json', transforms=valid_transform)

train_dataloader  = DataLoader(train_dataset, 
                               batch_size  = CFG.TRAIN.BATCH_SIZE, 
                               shuffle     = False,  
                               num_workers = CFG.TRAIN.WORKERS,
                               pin_memory  = CFG.TRAIN.PIN_MEMORY,
                               sampler     = make_sampler(train_dataset),
                               collate_fn  = collate_fn)
valid_dataloader  = DataLoader(valid_dataset, 
                               batch_size  = CFG.TRAIN.BATCH_SIZE, 
                               shuffle     = False, 
                               num_workers = CFG.TRAIN.WORKERS, 
                               pin_memory  = CFG.TRAIN.PIN_MEMORY,
                               collate_fn  = collate_fn)    

### Model

In [ ]:
model = MASK_MODEL(num_channel = CFG.DATA.N_CHANNEL, 
                   num_class   = CFG.DATA.N_CLASS,
                   min_size    = CFG.TRAIN.IMG_SIZE, 
                   max_size    = CFG.TRAIN.IMG_SIZE, 
                   image_mean  = CFG.DATA.N_CHANNEL * [CFG.TRAIN.IMG_MEAN],
                   image_std   = CFG.DATA.N_CHANNEL * [CFG.TRAIN.IMG_STD],
                   pretrained  = CFG.TRAIN.MASK.PRETRAINED)
model.load_state_dict(torch.load('mask_model/best.pth', map_location=CFG.TRAIN.DEVICE))
model.to(CFG.TRAIN.DEVICE)

optimizer = AdaBelief([p for p in model.parameters() if p.requires_grad], 
                      lr               = CFG.OPTIMIZER.MASK.LR,
                      eps              = CFG.OPTIMIZER.MASK.EPSILON,
                      weight_decay     = CFG.OPTIMIZER.MASK.WEIGHT_DECAY,
                      weight_decouple  = CFG.OPTIMIZER.MASK.WEIGHT_DECOUPLE,
                      rectify          = CFG.OPTIMIZER.MASK.RECTIFY,
                      print_change_log = False)

scheduler = CosineAnnealingWarmupRestarts(optimizer, 
                                          first_cycle_steps = CFG.SCHEDULER.MASK.FIRST_CYCLE_STEPS,
                                          warmup_steps      = CFG.SCHEDULER.MASK.WARMUP_STEPS,
                                          max_lr            = CFG.OPTIMIZER.MASK.LR,
                                          min_lr            = CFG.SCHEDULER.MASK.MIN_LR)

scaler = torch.cuda.amp.GradScaler()

In [ ]:
def train_one_epoch(epoch, dataloader, model, optimizer, scheduler, scaler, device):
    model.train()    
    loss_meter = AverageMeter()
    pbar = tqdm(dataloader, desc=f'Epoch [{epoch}]', leave=True)
    for images, targets in pbar:  
        optimizer.zero_grad()
        if scheduler is not None:
            scheduler.step(epoch+loss_meter.count/len(dataloader))

        images    = [image.to(device) for image in images]
        targets   = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast():
            loss_dict = model(images, targets)
            loss      = sum(loss for loss in loss_dict.values())
            loss_meter.update(loss.item(), n=1)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        pbar.set_postfix({'Lr': optimizer.param_groups[0]['lr'], 'Loss': loss_meter.avg})

### Training

In [ ]:
# Make Log File
log_name = str(datetime.now())[:19]
log_name = log_name.replace(' ', '-').replace(':', '-')
log_name = 'MASK_' + log_name + '.txt'
sys.stdout = open(log_name, 'w')

# Print CFG
print_time(message='START MASK', new_line=False)
print(CFG)

# Make Dir
os.makedirs(CFG.MODEL.MASK_FOLDER, exist_ok=True)

# Loop
best_score = 0
for epoch in range(50, CFG.TRAIN.PSEUDO_MASK.EPOCHS):
    # Train
    print_time(message=f'EPOCH = {epoch}')
    train_one_epoch(epoch, train_dataloader, model, optimizer, scheduler, scaler, CFG.TRAIN.DEVICE)
    
    # Validate
    true, pred = evaluate(model, valid_dataloader, device=CFG.TRAIN.DEVICE)
    score = print_mAP(true, pred, conf=CFG.TRAIN.MASK.CONF)
    
    # Last File Save
    model_path = os.path.join(CFG.MODEL.MASK_FOLDER, f'mk{epoch:03d}_{score:.4f}.pth')
    torch.save(model.state_dict(), model_path)
    
    # Best File Save
    if score > best_score:
        best_score = score
        model_path = os.path.join(CFG.MODEL.MASK_FOLDER, 'best.pth')
        torch.save(model.state_dict(), model_path)
        
print_time(message='END: MASK')